# Convolution Neural Network

In [1]:
import deeplake
import numpy as np

import tqdm
import random
import pandas as pd
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

C:\Users\limya\AppData\Local\Programs\Python\Python311\Lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.8.4) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [2]:
print(torch.__version__)

2.0.1+cpu


In [3]:
from dataprocessing import celebA_test_dataloader, celebA_train_dataloader, celebA_val_dataloader, adience_dataloader

Opening dataset in read-only mode as you don't have write permissions.


/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/adience



\

hub://activeloop/adience loaded successfully.



...

/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/celeb-a-test



|

hub://activeloop/celeb-a-test loaded successfully.



C:\Users\limya\AppData\Local\Programs\Python\Python311\Lib\site-packages\deeplake\integrations\pytorch\common.py:126: UserWarning: Decode method for tensors ['images'] is defaulting to numpy. Please consider specifying a decode_method in .pytorch() that maximizes the data preprocessing speed based on your transformation.
  warnings.warn(


In [4]:
class Levi_Hassner(nn.Module):
    def __init__(self,output=2,deformable=False) -> None:
        super().__init__()
        self.deformable=deformable

        self.layers=nn.Sequential(OrderedDict([
            # first convolutional layer
            ('conv1',nn.Conv2d(3, 96, 7, padding='valid', stride=4)),  # No padding
            ('relu1',nn.ReLU()),
            ('maxpool1',nn.MaxPool2d(3, stride=2)),  # Max pooling over a (3, 3) window with 2 pixel stride)
            ('lrn1',nn.LocalResponseNorm(size=5, k=2, alpha=10**(-4), beta=0.75)),

            # second convolutional layer
            ('conv2',nn.Conv2d(96, 256, 5, padding='same')), # Same padding
            ('relu2',nn.ReLU()),
            ('maxpool2',nn.MaxPool2d(3, stride=2)),  # Max pooling over a (3, 3) window with 2 pixel stride)
            ('lrn2',nn.LocalResponseNorm(size=5, k=2, alpha=10**(-4), beta=0.75)),

            # third convolutional layer
            ('conv3',nn.Conv2d(256, 384, 3, padding='same')),  # Same padding
            ('relu3',nn.ReLU()),
            ('maxpool3',nn.MaxPool2d(3, stride=2)),  # Max pooling over a (3, 3) window with 2 pixel stride)
            ('flatten',nn.Flatten()),

            ('fc1',nn.Linear(384*6*6, 512)), # input 384 * 6 * 6 = 13824, output 512
            ('relu4',nn.ReLU()),
            ('dropout1',nn.Dropout(0.5)),

            ('fc2',nn.Linear(512,512)),
            ('relu5',nn.ReLU()),
            ('dropout2',nn.Dropout(0.5)),
            
            ('fc3',nn.Linear(512,output)), # output = number of classes 
        ]))
        self.prob=nn.Softmax

    def forward(self,x):
        x=self.layers(x)
        prob=self.prob(x)
        return prob

In [5]:
gender_model = Levi_Hassner()
gender_model

Levi_Hassner(
  (lrn): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=valid)
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=13824, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=2, bias=True)
  (prob): Softmax(dim=None)
)

In [6]:
age_model = Levi_Hassner(output=8)
age_model

Levi_Hassner(
  (lrn): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=2)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv1): Conv2d(3, 96, kernel_size=(7, 7), stride=(4, 4), padding=valid)
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=same)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=13824, out_features=512, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=8, bias=True)
  (prob): Softmax(dim=None)
)

In [14]:
# Choose a loss function for training
# gender_model = Levi_Hassner(output=1)
loss_object = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gender_model.parameters(), lr=1e-3)
EPOCHS = 30

# Train step
def train_step(images, labels, optimizer, model):
    optimizer.zero_grad()
    predictions = model(images)
    loss = loss_object(predictions, labels)
    loss.backward()
    optimizer.step()
    return loss.item(), (predictions == labels).type(torch.float).mean().item()

def test_step(images, labels, model):
    predictions = model(images)
    t_loss = loss_object(predictions, labels)
    return t_loss.item(), (predictions == labels).type(torch.float).mean().item()

for epoch in range(EPOCHS):
        train_loss = 0.0
        train_accuracy = 0.0
        test_loss = 0.0
        test_accuracy = 0.0
        train_steps = 0
        test_steps = 0

        for images, age_labels, gender_labels in celebA_train_dataloader:
            l, a = train_step(images, gender_labels, optimizer, gender_model)
            train_loss += l
            train_accuracy += a
            train_steps += 1

        with torch.no_grad():
            for test_images, age_labels, gender_labels in celebA_val_dataloader:
                t_l, t_a = test_step(test_images, gender_labels, gender_model)
                test_loss += t_l
                test_accuracy += t_a
                test_steps += 1

C:\Users\limya\AppData\Local\Temp\ipykernel_16900\1010518302.py:43: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  prob = self.prob(fc3)


RuntimeError: 0D or 1D target tensor expected, multi-target not supported

# Transfer Learning

In [ ]:
for param in gender_model.parameters():
    param.requires_grad = False
for param in gender_model.fc3.parameters(): # unfreeze weights of last layer
    param.requires_grad = True

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gender_model.fc3.parameters(), lr=0.001)

# Training loop
for epoch in range(5):
    optimizer.zero_grad()
    outputs = resnet18(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}/5, Loss: {loss.item()}')